# analyze video(s)
Various cells for analyzing videos, groups of videos, etc...

In [1]:
# inits
from deepposekit.io import DataGenerator, VideoReader, VideoWriter
from .predict.utils import analyze_video, make_tracking_video
from deepposekit.io import VideoReader, VideoWriter, utils
from deepposekit.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import cv2
import sys
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
%load_ext autoreload
%autoreload 2


# settings
model_name = r'D:\github\fish-tracking\train\models\tank_model.h5'
skeleton = r'D:\github\fish-tracking\label\skeletons\tank_skeleton_3.csv'


ModuleNotFoundError: No module named '__main__.predict'; '__main__' is not a package

In [4]:
%pwd
from ..predict.utils import analyze_video

ValueError: attempted relative import beyond top-level package

## analyze single video
Analyze a single video, and make video with tracking overlaid.

In [2]:
# analyze single video

# settings
video = r'Z:\locker\ShareData\tank_for_rick\tank_vids\20201003_Hank\concatenated.avi'

# analyze video
model = load_model(model_name)
predictions = analyze_video(video, model, skeleton, batch_size=32)
make_tracking_video(video, skeleton, predictions=predictions)


analyzing video: Z:\locker\ShareData\tank_for_rick\tank_vids\20201003_Hank\concatenated.avi
11927/11927 [==============================] - 4090s 343ms/step
making tracking video: Z:\locker\ShareData\tank_for_rick\tank_vids\20201003_Hank\concatenated_tracking.avi


381648it [18:42, 340.03it/s]
